<a href="https://colab.research.google.com/github/ABmaxplunck/PyTorch-Codes-/blob/main/02_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Linear Regression one of the foundational algorithm of machine learning.
we will create a model that preducts crop yields for apples and oranges (target variables) by looking at the average temparature, rainfall and humidity (input variables or features) in a region.

In [25]:
import numpy as np
import torch

### Training data
The training data can be represented using 2 matrices: inputs and targets, each with one row per observation and one column per variable

In [26]:
  # input(temp,rainfall,humidity)
  
  inputs = np.array([[73,67,43],
                      [91,88,64],
                      [87,134,58],
                      [102,13,37],
                      [69,96,70]], dtype='float32')

In [27]:
# Targets(apples, oranges)
targets = np.array([[56,70],
                    [81,101],
                    [119,133],
                    [22,37],
                    [103,119]], dtype='float32')

we've seperated the input and target variables, because we'll operate on them seperately. Also, we've created numpy arrays, because this is typically how you would work with training data:read some CSV files as numpy arrays, do some processing and then convert them to Pytorch tensors as follows:

In [28]:
# convert input and targets to tensors 
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)


tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  13.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


### Linear regression model from scratch
The weights and biases (w11, w12,...., w23, b1 & b2) cann also be represented as matrices, initialized as random values. The first row of w and the first element of b are used to predict the first target variable i.e: yield of apples and similarly the second for oranges.

In [29]:
# weigts and biases 

w = torch.randn(2, 3, requires_grad=True) # it has 2 row and 3 col
b = torch.randn(2, requires_grad=True) # it has 2 row  
print(w)
print(b)


tensor([[ 0.6724, -1.3824, -0.5022],
        [-1.2477, -0.0517,  0.5013]], requires_grad=True)
tensor([-1.7851,  1.5429], requires_grad=True)


we can define the model as follows:


In [30]:
def model(x):
  return x @ w.t() + b

`@` represent the matrix multiplication in Pytorch and the `.t` method returns the transpose of a tensor

The matrix obtained by passing the input data into the model is a set of predictions for the target variables

In [31]:
# Generate predictions
preds = model(inputs)
print(preds)


tensor([[ -66.9168,  -71.4419],
        [ -94.3903,  -84.4573],
        [-157.6593,  -84.8507],
        [  30.2472, -107.8426],
        [-123.2553,  -54.4141]], grad_fn=<AddBackward0>)


let's compare the predictions of our model with the actual targets

In [32]:
# compare with targets 
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


you can see that there's a huge differnce between the predictions of our model and the actual values of the target variables. Obviously this is because we've initialized our model with random weights and biases and we can't expect it to just work 

### Loss Function
Before we improve our model, we need a way to evaluate how well our model is performing. we can compare the model's precitions with the actual targets, using the following method:

* calculate the difference between the two matrices (preds and targets)
* square all elements of the difference matrix to remove negative values 
* calculate the average of the elements in the resulting matrix

The result is a single number known as the `mean squared error (MSE)`

In [33]:

diff = preds - targets
diff_sqr = diff * diff # element wise multiplication 
torch.sum(diff_sqr) /  diff.numel()  # using .numel method to get the num of element 

tensor(32658.1191, grad_fn=<DivBackward0>)

In [34]:
# MSE loss function

def mse(t1,t2):
  diff = t1 -t2
  return torch.sum(diff * diff)/diff.numel()

`torch.sum` returns the sum of all the elements in a tensor and the `.numel` method returns the number of elements in a tensor. Let's compute the mean squared error for the current predictions of our model 

In [35]:
# compute loss
loss = mse(preds, targets)
print(loss)



tensor(32658.1191, grad_fn=<DivBackward0>)


here's how we can interpret the result: On average each element in the prediction differs from the actual target by about 138(square root of the loss 19044). And that's pretty bad, considerting the numbers we are trying to predict are themselves in the range 50-200. Also, the result is called the loss, because it indicates how bad the model is at preciting the target variables. Lower the loss better the model 

### Compute gradients
With PyTorch, we can automatically compute the gradient or derivative of the loss w.r.t to the weights and biases, because they have requires_grad set to True

In [36]:
# Compute gradients 

loss.backward()

The gradients are stored in the `.grad` property of the respective tensors. Note that the derivative of the loss w.r.t the weights matrix is itself a matrix, with the same dimensions 

In [37]:
# Gradients for weights
print(w)
print(w.grad)

tensor([[ 0.6724, -1.3824, -0.5022],
        [-1.2477, -0.0517,  0.5013]], requires_grad=True)
tensor([[-12754.6426, -16471.0840,  -9617.8750],
        [-14578.8799, -14703.9111,  -9616.9551]])


`w` is a weight matrix or tensor.
I have each corresponding weights and derivative of the loss

from `print(w)` we get losses which are w11, w12 etc etc 

from `print(w.grad)` we get derivative of the loss element. these are d loss by dw11, this is d loss by dw12 etc etc 

In [38]:
# For bias 
print(b)
print(b.grad)

tensor([-1.7851,  1.5429], requires_grad=True)
tensor([-158.5949, -172.6013])




> Indented block


The loss is a `quadratic function` of our weights and biases, and our objective is to find the set of weights where the loss is the lowest. If we plot a graph of the loss w.r.t any individual weight or bias element, it will look like figureshown below. A key insight from calculus is that the gradient indicates the rate of change of the loss, or the slope of the graph or loss function w.r.t the weights and biases

If a gradient element is `positive` 
* increasing the element's value slightly will increase the loss

* decreasing the element's value slightly will decrease the loss

If the gradient element is `negative` 
* increasing the element's value slightly will decrease the loss

* decreasing the element's value slightly will increase the loss



Before we proceed we reset the gradients to zero by calling `.zero_()` method. we need to do this, because PyTorch accumulates gradients i.e: the next time we call `.backward` on the loss, the new gradient values will get added to the existing gradient values, which may lead to unexpected results 

In [39]:

w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


### Adjust weights and biases using gradient descent
we'll reduce the loss and improve our model using the gradient descent optimization algorithm, which has the follwing steps:
1. Generate predictions
2. Calculate the loss
3. compute gradients w.r.t the weights and biases
4. Adjust the weights by subtracting a small quantity proportional to the gradient 
5. Reset the gradients to zero

In [40]:
# Generate predictions

preds = model(inputs)
print(preds)

tensor([[ -66.9168,  -71.4419],
        [ -94.3903,  -84.4573],
        [-157.6593,  -84.8507],
        [  30.2472, -107.8426],
        [-123.2553,  -54.4141]], grad_fn=<AddBackward0>)


note that the predictions are same as before, since we haven't made any changes to our model. the same hols true for the loss and gradients

In [41]:
# calculate the loss

loss = mse(preds, targets)
print(loss)

tensor(32658.1191, grad_fn=<DivBackward0>)


In [42]:
# compute gradients

loss.backward()
print(w.grad)
print(b.grad)

tensor([[-12754.6426, -16471.0840,  -9617.8750],
        [-14578.8799, -14703.9111,  -9616.9551]])
tensor([-158.5949, -172.6013])


finally we update the weights and  biases using the gradients computed above 

In [43]:
# adjust weights and reset gradients 

with torch.no_grad():  # torch.no_grad() is way of telling pytorch that hey I'm done with my gradient calculations now let me use the gradients  
  w -= w.grad * 1e-5    # when I am running this operations don't track gradient related work 
  b -= b.grad * 1e-5
  w.grad.zero_() # reset the gradient back to zero
  b.grad.zero_()


a few things to note above:
* we use torch.no_grad to indicate PyTorch that we shouln't track, calculate or modify gradients while updating the weights and biases
* we multiply the gradients with a really small number (10^-5 in this case), to ensure that we don't modify the weights by a really large amount, since we only want to takse a small step in the downhill direction of the gradient. This number is called the learning rate of the algorithm 
* after we have updated the weights, we reset the gradients back to zero to avoid affecting any future computations 

Let's take a look at the new weights and biases 

In [44]:
print(w)
print(b)

tensor([[ 0.7999, -1.2177, -0.4060],
        [-1.1019,  0.0954,  0.5975]], requires_grad=True)
tensor([-1.7835,  1.5446], requires_grad=True)


with the new weights and biases, the model should have lower loss

In [45]:
# calculate loss
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(23074.2910, grad_fn=<DivBackward0>)


we have already achieved a significant reduction in the loss, simply by adjusting the weights and biases slightly using gradient descent 

## Train for multiple epochs

To reduce the loss further we can repeat the process of adjusting the weights and biases using the gradients multiple times. Each iteration is called an epoch. let's train the model for 100 epochs

In [46]:
# Train for 100 epochs

for i in range (100):
  preds = model(inputs) # get the predictions passing the inputs 
  loss = mse(preds, targets) # get the loss by passing the predctions and targets into the mse function
  loss.backward() # calculcate the gradients 
  with torch.no_grad(): #telling pytorch hey stop tracking gradient for a sec
    w -= w.grad * 1e-5 # here 1e-5 is learning rate which is called hyper parameter in ML
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()


once again, let's verify that the loss is now lower:

In [47]:
# calcualte lose
preds = model(inputs)
loss = mse(preds, targets)
print(loss)


tensor(279.1749, grad_fn=<DivBackward0>)


As you can see, the loss is now much lower than what we started out with. Let's look at the model's predictions and compare them with the targets 

In [48]:
# precitions 
preds

tensor([[ 68.0919,  73.8266],
        [ 87.8060, 105.4909],
        [103.5443, 130.2961],
        [ 62.1459,  25.4397],
        [ 79.2656, 122.0900]], grad_fn=<AddBackward0>)

In [49]:
# targets 
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

The prediction are now quite close to the target variables, and we can get even better results by training for a few more epochs.

At this point, we can save our notebook and upload it to the git for future reference and sharing 
